<a target="_blank" href="https://colab.research.google.com/github/trainocate-japan/Machine-Learning-and-Deep-Learning-Hands-on/blob/main/exercise/2_線形回帰/2-1-2_線形回帰による中古マンションの価格予測②.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 2-1-2_線形回帰による中古マンションの価格予測（続き）

## ライブラリのインポート

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# matplotlibで日本語表示するための設定
!pip install japanize_matplotlib | tail -n 1
import japanize_matplotlib

## データの準備

#### データを取り込む

In [ ]:
# csvファイルを読み込みます
totsuka_used = pd.read_csv("totsuka_used.csv")

## 予測精度の改善① 説明変数を増やしてみる

#### 変数の相関（関係性）を確認する
説明変数を増やす前に特に有用性が高そうだと考えられる変数の関係性を確認します。
seabornライブラリには、変数の相関関係を簡単にプロットすることのできるペアプロットのメソッドが用意されています。

pairplotメソッドの使い方は引数に関係性を見たい変数を指定するだけです。

In [ ]:
sns.pairplot( totsuka_used[['取引価格','築年数','最寄駅距離（分）','面積（㎡）']])

特に着目すべきは一番上段の取引価格と他の変数の関係性です。<br>
築年数や最寄駅からの距離には何となく値が増えるほど価格が下がる右肩下がりな傾向があります。<br>
対して、面積は値が増えるほど価格が上がる傾向があるように見えます。

#### 説明変数を増やす
説明変数に築年数、最寄駅距離（分）、面積（㎡）を設定します。

In [ ]:
train_x = totsuka_used[['築年数','最寄駅距離（分）','面積（㎡）']]

In [ ]:
train_y = totsuka_used['取引価格']

新たな説明変数で、モデルを学習させ、評価します。

In [ ]:
# モデルを定義する
model = LinearRegression()

In [ ]:
# 学習
model.fit(train_x, train_y)

In [ ]:
# 傾きを求める
# 順番に築年数、最寄り駅距離（分）、面積（㎡）
model.coef_

In [ ]:
# 切片を求めるにはintercept_プロパティを使う
model.intercept_

In [ ]:
# 精度を確認してみる
model.score(train_x, train_y)

## 予測精度の改善② カテゴリ変数を説明変数として使う
追加した説明変数は予測に有用だったようで、予測精度を向上させることができました。<br>
さらに、物件のある地区や間取りの情報を説明変数として用いれば精度が向上するかもしれませんが、この二つの列は連続値ではなく、離散値のためそのままでは計算に用いることができません。

そこで、この変数を説明変数として用いることができるようにダミー変数化します。

*   まずは二つの変数についてどのような種類の値が入っているか確認してみます。<br>
DataFrameのvalue_counts()メソッドで値を種類ごとにカウントすることができます。
*   サンプルの少ないカテゴリもあるので、ノイズになる可能性もありそうですが、今回はそのまま使用してみます。（あまり関係性のないデータが入ることで誤った傾向が学習されてしまうようなものをノイズといいます）

In [ ]:
# value_countsで値の種類ごとに数を数える
totsuka_used['地区名'].value_counts()

In [ ]:
totsuka_used['間取り'].value_counts()

#### ダミー変数化の実行
- pd.get_dummiesメソッドを使います
- 引数にダミー変数化したい変数を書きます

In [ ]:
# 試しに地区名のみダミー変数化してみる
pd.get_dummies(totsuka_used['地区名']).head(3)

必要な説明変数をすべてダミー変数化する<br>
もとから連続値になっているものはそのまま選択されるので、必要な列をすべて指定する

In [ ]:
train_x = pd.get_dummies( totsuka_used[['築年数','最寄駅距離（分）','面積（㎡）','地区名', '間取り']])

train_xの中身を確認してみます

In [ ]:
train_x.head()

モデルを再度学習させて、評価します

In [ ]:
model.fit(train_x, train_y)

In [ ]:
# 精度を確認してみる
model.score(train_x, train_y)

## 訓練データと検証データに分割して評価する

In [ ]:
# scikit-learnで提供される、訓練データと検証データを分けるライブラリをインポートする。
from sklearn.model_selection import train_test_split

In [ ]:
# 訓練データと検証データに分割(70%を訓練用に使用)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, train_size=0.7, test_size=0.3, random_state=0)

In [ ]:
# データ分割後の訓練データの件数を確認する
train_x.shape

In [ ]:
# データ分割後の検証データの件数を確認する
val_x.shape

分割したデータで訓練データ、検証データそれぞれに対して精度を求めてみます。

In [ ]:
# 訓練データを使用して学習
model.fit(train_x, train_y)

In [ ]:
# 訓練データに対しての精度を確認してみる
model.score(train_x, train_y)

In [ ]:
# 検証データに対しての精度を確認してみる
model.score(val_x, val_y)

訓練データと検証データに分けそれぞれ精度を確認すると、検証データのほうが低く精度が出ています。<br>
これは自然なことで、未知のデータである検証データのほうが予測は難しくなります。

## （参考）モデルの保存とロード
機械学習モデルで実際に予測をする際は毎回プログラムを動かして学習をさせるわけにはいかないので、通常は学習済みモデルを保存します。<br>
保存方法はいくつかありますが、scikit-learnのモデルを保存する場合はpickleというライブラリを使用して以下のように保存とロードを行うことができます。

In [ ]:
# pickleのインポート
import pickle

# モデルの保存
with open('model.pickle', mode='wb') as f:
  pickle.dump(model, f)

In [ ]:
# 保存されたファイルを確認
!dir

In [ ]:
# モデルのロード
with open('model.pickle', mode='rb') as f:
  load_model = pickle.load(f)

In [ ]:
# ロードされたモデルで訓練データを予測して、最初の5件を確認してみる。
# ※訓練データがシャッフルされているので、抽出されるサンプルは変わっています。
load_model.predict(train_x)[:5]

このノートブックは以上です。